In [1]:
%matplotlib inline
%load_ext dotenv
%dotenv env.sh
%run source/utils.py
import matplotlib
import matplotlib.pyplot as plt
from skimage import exposure
import pandas as pd
import numpy as np
import cytokit
from cytokit.image.ops import describe
import os
import os.path as osp
import tqdm

In [2]:
df = pd.read_csv('experiments.csv')
df.head()

,name,conf,dir,grid,n_ch,n_tiles,n_z,cond,rep,z_pitch
0,00.0mT-np-0uMsta-XY01,experiment.yaml,untreated-noTcells-2,XY01,4,33,8,00.0mT-np-0uMsta,1,200
1,00.0mT-np-0uMsta-XY02,experiment.yaml,untreated-noTcells-2,XY02,4,33,8,00.0mT-np-0uMsta,2,200
2,00.0mT-np-0uMsta-XY03,experiment.yaml,untreated-noTcells-2,XY03,4,33,7,00.0mT-np-0uMsta,3,200
3,02.5mT-np-0uMsta-XY01,experiment.yaml,025mioTcells-nopeptide,XY01,4,33,9,02.5mT-np-0uMsta,1,200
4,02.5mT-np-0uMsta-XY02,experiment.yaml,025mioTcells-nopeptide,XY02,4,33,8,02.5mT-np-0uMsta,2,200


In [4]:
export_dir = '/lab/data/spheroid/ilastik/mc38/training/20x-s25-spheroid-body-02/hoechst-maxz'
export_dir

'/lab/data/spheroid/ilastik/mc38/training/20x-s25-spheroid-body-02/hoechst-maxz'

In [5]:
np.random.seed(0)

def export_images(exp_name, images, export_dir, n_images=5):
    idx = np.random.choice(np.arange(len(images)), size=n_images, replace=False)
    images = [images[i] for i in idx]
    for img, ti in images:
        img = transform.rescale(img, .5, anti_aliasing=True, multichannel=False, mode='constant', preserve_range=True)
        path = osp.join(export_dir, '{}-T{:03d}-X{:03d}-Y{:03d}.tif'.format(exp_name, ti.tile_index+1, ti.tile_x+1, ti.tile_y+1))
        if not osp.exists(osp.dirname(path)):
            os.makedirs(osp.dirname(path))
        assert 0 <= img.min() <= img.max() <= 1
        img = exposure.rescale_intensity(img, in_range=(0, 1), out_range=np.uint8).astype(np.uint8)
        # Save image with original contrast, but downscaled to 25% (50% here and 50% in the original acquisition) with 8-bit depth
        ck_io.save_image(path, img)

for i, r in tqdm.tqdm(df.iterrows(), total=len(df)):
    config = get_config(r)
    raw_dir = osp.join(os.environ['EXP_GROUP_RAW_DIR'], r['dir'], r['grid'])
    images = list(get_channel_maxz_images(config, raw_dir, 'HOECHST'))
    export_images(r['name'], images, export_dir)

100%|██████████| 33/33 [27:39<00:00, 51.19s/it]
